In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w4p1/submit.csv
/kaggle/input/2022-ai-w4p1/train.csv
/kaggle/input/2022-ai-w4p1/test.csv


In [2]:
import torch
import torch.optim as optim
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [3]:
train = pd.read_csv('../input/2022-ai-w4p1/train.csv')
test = pd.read_csv('../input/2022-ai-w4p1/test.csv')
submit = pd.read_csv('../input/2022-ai-w4p1/submit.csv')

In [4]:
trainX = train.drop('Diabetes', axis=1)
trainY = train['Diabetes']

In [5]:
# DataFrame 말고 배열 형태로 넣어줘야함

x_train = torch.FloatTensor(trainX.to_numpy())
y_train = torch.FloatTensor(trainY.to_numpy())
x_test = torch.FloatTensor(test.to_numpy())

In [6]:
print(x_train.shape)

torch.Size([537, 8])


In [7]:
W = torch.zeros((x_train.shape[1],1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.01)

hypothesis = torch.sigmoid(torch.matmul(x_train, W)+b)

In [8]:
W = torch.zeros((x_train.shape[1],1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.0001)

np_epochs = 1000000
for epoch in range(np_epochs+1) :
    # 가설
    hypothesis = torch.sigmoid(torch.matmul(x_train, W)+b)
    # cost
    loss = torch.nn.BCELoss()
    cost = loss(hypothesis, y_train.unsqueeze(1))
    # cost로 update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100000 == 0 :
        print(epoch, cost.item())
    

0 0.6931473016738892
100000 0.6032845377922058
200000 0.5899330377578735
300000 0.5780060887336731
400000 0.5672978758811951
500000 0.5577350854873657
600000 0.5491641163825989
700000 0.5414417386054993
800000 0.5345455408096313
900000 0.5283812880516052
1000000 0.5227400660514832


In [9]:
hypothesis = torch.sigmoid(torch.matmul(x_test, W)+b).detach()
submit['Diabetes'] = hypothesis >= torch.FloatTensor([0.5])

In [10]:
submit = submit.replace({'Diabetes' : True}, 1) 
submit = submit.replace({'Diabetes' : False}, 0) 

In [11]:
submit

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
226,226,1
227,227,0
228,228,0
229,229,0


In [12]:
submit.to_csv('submit3.csv', index=False)